<a href="https://colab.research.google.com/github/palash04/Artificial-Intelligence/blob/master/NN_Series/_PytorchNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [8]:
# create fully connected layer
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN,self).__init__()
    self.hidden_size = 50
    self.fc1 = nn.Linear(input_size,  self.hidden_size)
    self.fc2 = nn.Linear(self.hidden_size, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x), dim=1)
    
    return x

In [9]:
# testing NN
model = NN(784,10)
x = torch.rand((64,784))
output = model(x)
print (output.shape)  # must be 64 x 10

torch.Size([64, 10])


In [10]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [12]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [13]:
# Initialize the Network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [17]:
# Train Network
for epoch in range(1,num_epochs+1):
  train_loss = 0.0
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda is possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # get to correct shape
    # data is in the form of (64 x 1 x 28 x 28)
    # we need data to be in the form of (64 x 784)
    data = data.reshape(data.shape[0], -1)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()   # store gradients to zero for each batch
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

    train_loss += loss.item()
  print (f'Epoch: {epoch}/{num_epochs} \t Training Loss: {train_loss/(len(train_loader)):.6f}')


Epoch: 1/10 	 Training Loss: 1.532268
Epoch: 2/10 	 Training Loss: 1.523150
Epoch: 3/10 	 Training Loss: 1.515881
Epoch: 4/10 	 Training Loss: 1.510483
Epoch: 5/10 	 Training Loss: 1.506250
Epoch: 6/10 	 Training Loss: 1.501982
Epoch: 7/10 	 Training Loss: 1.499164
Epoch: 8/10 	 Training Loss: 1.496359
Epoch: 9/10 	 Training Loss: 1.494027
Epoch: 10/10 	 Training Loss: 1.491571


In [18]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print ('Checking accuracy on training data')
  else:
    print ('Checking accuracy on test data')
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)

      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, pred = scores.max(1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0)

    print (f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100 :.2f}')
  
  model.train()

In [19]:
check_accuracy(train_loader, model)
check_accuracy(test_loader,model)

Checking accuracy on training data
Got 58446/60000 with accuracy 97.41
Checking accuracy on test data
Got 9629/10000 with accuracy 96.29
